In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

In [6]:
train_df = pd.read_csv(Path('Resources/2019loans.csv', index_col = False))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))

train_df = train_df.iloc[: , 2:]
test_df = test_df.iloc[: , 2:]

train_df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,target,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,13375,0.1797,483.34,MORTGAGE,223000.0,Not Verified,low_risk,n,29.99,0,...,100.0,50.0,0,0,577150,122018,32000,170200,N,N
1,21000,0.1308,478.68,MORTGAGE,123000.0,Source Verified,low_risk,n,11.26,2,...,85.0,33.3,0,0,132750,27896,15900,35398,N,N
2,20000,0.1240,448.95,MORTGAGE,197000.0,Source Verified,low_risk,n,11.28,0,...,85.7,33.3,0,0,628160,114043,22600,90340,N,N
3,3000,0.1240,100.22,RENT,45000.0,Not Verified,low_risk,n,18.08,0,...,100.0,16.7,1,0,42006,20761,19900,15406,N,N
4,30000,0.1612,1056.49,MORTGAGE,133000.0,Source Verified,low_risk,n,27.77,0,...,100.0,66.7,0,0,283248,109056,79500,58778,N,N


In [7]:
# Separate the target column (y) from the predictive features (X) in both our train and test dfs
X_train = train_df.drop("target", axis=1).copy()
X_test = test_df.drop("target", axis=1).copy()
y_train = train_df["target"].copy()
y_test = test_df["target"].copy()

In [8]:
# Convert categorical data to numeric and separatX_train = pd.get_dummies(X_train)
X_train = pd.get_dummies(X_train)
X_train.head()
X_test = pd.get_dummies(X_test)
X_test.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N
0,40000,0.0819,814.70,140000.0,19.75,0,1,18,0,9471,...,0,0,1,0,1,1,0,1,0,1
1,6000,0.1524,208.70,55000.0,11.52,2,0,8,0,1280,...,0,0,1,0,1,1,0,1,0,1
2,3600,0.1695,128.27,42000.0,6.74,0,0,6,0,4757,...,0,0,1,0,1,1,0,1,0,1
3,20000,0.1524,478.33,100000.0,12.13,0,2,7,0,12731,...,0,0,1,0,1,1,0,1,0,1
4,3600,0.1240,120.27,50000.0,16.08,0,3,6,0,10413,...,0,0,1,0,1,1,0,1,0,1


# Hypothesis
I will be comparing two different machine learning models to try and find which can predit with higher accuracy the risk of a loand given to someone. The two models being compared are: Logistic regression and random forests classifier (RFC). I believe that the RFC will underperform compared to the Logistic Regression because we have a lot of varied data and this can lead to overfitting in the model. I belive the logistic regression model will perform better because most of these columns appear to be correlated with one another.

In [9]:
# First, identify any that are in one df and not the other.
cols_to_add = set(X_train.columns) ^ set(X_test.columns)

# Loop through each missing column and add it to X_test with all 0s
for col in cols_to_add:
    X_test[col] = 0

# Set the order of the columns in X_test to match the order of columns in X_train
X_test = X_test[X_train.columns]

# Encode the target column with 1s and 0s (both train and test)
target_encoder = LabelEncoder().fit(y_train)
y_train = target_encoder.transform(y_train)
y_test = target_encoder.transform(y_test)

In [10]:
# Check the shapes to make sure X_train and X_test have the same number of columns (same with y_train/y_test)
# Check that X_train and y_train have the same number of rows (same with X_test/y_test)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(12180, 92)
(4702, 92)
(12180,)
(4702,)


In [11]:
# Train the Logistic Regression model on the unscaled data and print the model train and test score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

lr = LogisticRegression()
lr.fit(X_train, y_train)
print(lr.score(X_test, y_test))
print(lr.score(X_train, y_train))

0.5168013611229264
0.6507389162561577


C:\Users\Xander\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [12]:
# Train a Random Forest Classifier model and print the model score
# Train a Random Forest Classifier model and print the model train and test score
rf = RandomForestClassifier(n_estimators=350, max_depth=3)
rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))
print(rf.score(X_train, y_train))

0.582092726499362
0.7207717569786535


In [13]:
# Scale the data
scaler = StandardScaler().fit(X_train)

# Use the scaler on X_train and X_test
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [19]:
# Train the Logistic Regression model on the scaled data and print the model train/test score
lr.fit(X_train_scaled, y_train)
print(f'Test Score: {lr.score(X_test_scaled, y_test)}')
print(f'Train Score: {lr.score(X_train_scaled, y_train)}')


Test Score: 0.767333049766057
Train Score: 0.7078817733990148


C:\Users\Xander\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [16]:
# Train a Random Forest Classifier model on the scaled data and print the model train/test score
rf.fit(X_train_scaled, y_train)
print(f'Test Score: {rf.score(X_test_scaled, y_test)}')
print(f'Train Score: {rf.score(X_train_scaled, y_train)}')

rf.feature_importances_

Test Score: 0.620374308804764
Train Score: 0.7278325123152709


array([8.62377273e-03, 1.07699046e-01, 1.86390669e-02, 9.73065130e-04,
       4.10522599e-04, 1.57457009e-04, 4.47978340e-03, 3.31518134e-05,
       9.82702667e-06, 1.67326275e-03, 1.36613887e-04, 2.81489961e-02,
       2.93079435e-02, 9.26981448e-02, 9.06130885e-02, 7.03152106e-02,
       1.23547069e-01, 6.64195000e-02, 0.00000000e+00, 0.00000000e+00,
       2.24940022e-01, 8.13284586e-06, 0.00000000e+00, 0.00000000e+00,
       2.65409786e-05, 9.73597646e-04, 1.36872999e-03, 2.59822221e-04,
       1.26108490e-04, 6.05797544e-04, 6.61776629e-04, 6.16961218e-04,
       2.69886674e-03, 1.04670553e-03, 2.48416531e-03, 9.07463862e-03,
       2.84819995e-03, 6.94745366e-03, 8.81182842e-04, 1.47232855e-04,
       4.75728282e-03, 8.31817464e-03, 1.48688047e-03, 9.52229615e-03,
       2.73442535e-03, 0.00000000e+00, 0.00000000e+00, 3.21725400e-03,
       5.91481919e-03, 3.06546591e-03, 1.42692185e-03, 2.50423453e-03,
       2.93735105e-03, 6.35981539e-03, 3.61485344e-04, 6.57566831e-04,
      

# Conclusion

My hypothesis was confirmed after fitting the models. While the training accuracy was lower on this fitting, the test predictions had a  higher probability of being correct in guessing. The scaled data had a higher score than the unscaled data and this would make sense. The values in the data set have different weights and scaling them helps to compare them on a level playing field. The model I would use is a scaled Logistic regression model.
